# TO DO 
- Formulate a general format for the data set 
- Update the preprocessing properly 
- A trip is a class 
- A schedule is also a class which hold the instances of trip 


Schedule (df, start_leg, no_of_trips, etc..)

Trip (steering_duty, spread_over, ...)

### SET UP AND IMPORTS 

In [81]:

# SET UP AND IMPORTS 
import pandas as pd 
import datetime
import time
import random

df = pd.read_excel('dataset/processed/psl.xlsx').reset_index(drop=True)

# Adding the Terminal Gap and Duty only for processing 
df['Terminal Gap'] = [0] * len(df)
df['Duty'] = [0] * len(df)

START_PLACE = 'PSL'
#start_legs contain the legs with departure place as start_place
start_legs = df[df['Departure Place'] == START_PLACE].sort_values('Departure Time', ascending=True)
start_legs.reset_index(drop=True,inplace=True)
# setting the initial running time of start legs as its running time itself
start_legs['Duty'] = start_legs['Running Time'] 
#main_legs contains all other legs 
# main_legs = df[df['Departure Place'] != START_PLACE]
# MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
MIN_DUTY = 7 * 3600 #setting the minimum duty as 7:00 hrs + 15min sign in  and 15 min sign out = 07:30 hrs
# The ideal duty is 7:30 hrs with a 15 min break at the start and end . Then total 8:00 
# MAX_DUTY = 30600 # setting the maximum duty as 8:30 hrs
MAX_DUTY = 9 * 3600  # setting the maximum duty as 09:00 hrs + 15min sign in  and 15 min sign out = 09:30 hrs
# MAX_DUTY_HOURS = 9
# MAX_DUTY_MINUTES = 15

# MAX_DUTY = (MAX_DUTY_HOURS * 60 + MAX_DUTY_MINUTES) * 60

MAX_SPREAD_OVER = 43200 # setting maximum spread over to 12 hrs 

MIN_TERMINAL_GAP = 300
MAX_TERMINAL_GAP = 900

BREAK = 1800 # break time of 30 min to avoid 5hrs continuous working
IS_BREAK = False # setting the break status as false by default 

MAX_SPLIT = 3600 # Setting the maximum split time as 60 min 
MIN_TEMPSET_SIZE = 2 # define the minimum number of legs in a temp_set

BREAK_LIMIT_DUTY = 18000 # 5hrs of continuous duty  
MIN_LIMIT_DUTY = 12600 # After 3.5 hrs of duty we can try entering the 30 min break

# FOR THE PURPOSE OF MEASURING THE PERFORMANCE 
success = 0 # Not necessary , just to count the number of start legs taken 
trips_count = 0 # count the number of trips 
exe_generateTempSet  = 0
exe_popTempSet  = 0
# start_legs

### SUPPORTING FUNCTIONS 

#### General Functions 


In [82]:
def secToTime(seconds):
    hrs, rem = divmod(seconds, 3600)
    mint, sec = divmod(rem , 60)
    return datetime.time(hour=hrs, minute=mint, second= sec)

#### Generate Tempset 
- Below function creates the temset for a particular leg in the stack . It checks for the needed criteria and then creates the set of legs that are possibly gives us a feasible solution 

In [83]:
# SUPPORTING FUNCTIONS
def generateTempSet(leg, mode = 0) -> pd.DataFrame:
    # mode 0 means normal trips and mode 1 means consider 30 min break also
    # FN LOGIC IS NOT CORRECT , NEED TO CREATE THE TEMP SET ONLY WITH THE CRITERIA
    # SORTING THEM AFTER CREATING THE ENTIRE TEMPSET IS NOT OPTIMAL
    global exe_generateTempSet
    x = time.time()

    # Creating a copy of the main legs.Can be limited with steering duty remaining 
    temp_set_df = df[(df['Departure Place'] == leg['Arrival Place'])
                  & (df['Departure Time'] > leg['Arrival Time'])]
    temp_set_df = temp_set_df.sort_values(
        'Departure Time', ascending=True).reset_index(drop=True)
    temp_set_df['Terminal Gap'] = temp_set_df['Departure Time'] - leg['Arrival Time']
    # Sorting and resetting index
    # '''BELOW LINE MAY BE NOT NECESSARY'''
    # display(temp_set)
    temp_set = pd.DataFrame()
    # THERE ARE CHANCES THAT THE TEMP SET MAY BE EMPTY , SO WE ARE TRYING FOR 
    #  THE NEXT LEVEL SEARCHING
    if mode == 0:
        min_terminal_gap = MIN_TERMINAL_GAP
        terminal_gap = MAX_TERMINAL_GAP
        ADD_ON = 900 # 15 min
    else :
        min_terminal_gap = BREAK
        terminal_gap = BREAK + MAX_TERMINAL_GAP
        ADD_ON = 900 # 15 min
   
    # NEED TO OPTIMIZE BELOW LOOP AS EVERY TIME IT LOOPS THROUGH THE ENTIRE LEGS TILL
    # THE SPECIFIED TERMINAL GAP , CAN USE EXTEND OR CONCAT TO ITERATIVELY ADDING UP THE 
    # TEMP SET  
    while len(temp_set) < MIN_TEMPSET_SIZE and terminal_gap < MAX_SPLIT: # 1 hr split
        temp_set = temp_set_df[
            (temp_set_df['Terminal Gap'] >= min_terminal_gap) & (
                temp_set_df['Terminal Gap'] < terminal_gap)  # |
            # (temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)
        ]
        terminal_gap += ADD_ON
    # display(temp_set)
    # print(f"Type of gap = {type(temp_set['Terminal Gap'][0])}")
    # temp_set['Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap']
    temp_set.loc[:, 'Duty'] = leg['Duty'] + temp_set['Running Time'] # should bee run for all legs
    # temp_set.loc[temp_set['Terminal Gap'] < MAX_TERMINAL_GAP, 'Duty'] += temp_set.loc[temp_set['Terminal Gap'] < MAX_TERMINAL_GAP, 'Terminal Gap']
    mask = temp_set['Terminal Gap'] < MAX_TERMINAL_GAP
    temp_set.loc[mask, 'Duty'] += temp_set.loc[mask, 'Terminal Gap']
    y = time.time()
    exe_generateTempSet += (y - x) 
    temp_set.reset_index(drop=True, inplace=True)
    return temp_set

In [84]:
# df = pd.DataFrame()
# df['first'] = [1, 2, 3]
# df['second'] = [4, 5, 6]
# df['result'] = 100 + df['first'] + df['second']
# df

#### Pop Tempset 
- This function pops the currently visited leg from the corresponding tempset, then the tempset will always contain the possible legs from a given leg that satisfies the criteria needed   

In [85]:
def popTempSet(temp_set) -> pd.DataFrame:
    global exe_popTempSet
    x = time.time()
    temp_set.drop(0, inplace=True)
    # print("popTempSet")
    # display(temp_set)
    temp_set.reset_index(inplace=True)
    temp_set.drop('index',axis= 1, inplace=True)
    y = time.time()
    exe_popTempSet += (y - x)
    return temp_set

#### Display Trip
- Returns the trip once the stack is satisfying all the criteria 

In [86]:
def displayTrip(stack):
    global trips_count
    frame = pd.DataFrame()
    for x in stack:
        frame = pd.concat([frame, (x['current_leg'].to_frame()).T], ignore_index=True)
    frame['Terminal Gap'] = frame['Terminal Gap'].shift(-1)

    #Calculating the steering duty 
    duty = secToTime(frame['Duty'].iloc[len(frame) - 1])
    spread_over = secToTime((stack[-1]['current_leg'])['Arrival Time'] - (stack[0]['current_leg'])['Departure Time'])
    frame = frame.drop('Duty', axis=1)
    trips_count += 1
    return [frame, duty, spread_over]

# def seconds_to_time(df, columns):
#     df[columns] = df[columns].apply(lambda x: pd.to_datetime(x, unit='s').dt.strftime('%H:%M:%S'))


In [87]:
# check_df = pd.DataFrame(
#     {'Sl No.': [1, 124, 313, 641, 336, 695],
#      'Departure Time': [13200, 13800, 18300, 22500, 29400, 36300],
#      'Departure Place': ['PSL', 'KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV'],
#      'Route of Operation': ['NH',
#                             'NH',
#                             'NH-UDA',
#                             'UDA',
#                             'NH',
#                             'PVR-VZM-BYPASS-TVM-NH-KLKV'],
#      'Arrival Place': ['KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV', 'PSL'],
#      'Arrival Time act': [13500, 17700, 21900, 28800, 36000, 48900],
#      'KM': [3.5, 33.7, 32.0, 39.0, 40.0, 81.5],
#      'Running Time': [300, 3900, 3600, 6300, 6600, 12600],
#      'Terminal Gap': [300, 600, 600, 600, 300, None]}
# )

#### Remove Legs
- Once a leg is taken for a trip , it should be removed so that it will not be considered for the next round of iterations  

In [88]:
def removeLegs(trip) -> None:
    global success
    for i in range(len(trip)):
        if trip.iloc[i]['Departure Place'] == START_PLACE:
            # remove from both df and start_legs 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            start_legs.drop((start_legs[start_legs['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            success += 1
        else:
            # remove only from df 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)

#### Backtracking 

In [89]:
def backtrack(stack):
    #  If temp_set is empty , it means that we cannot go to any other places from the 
    # last leg , so we can pop the last leg in the stack  
    stack.pop()
    # After popping stack top , there are also chances that the temp_set of top elements 
    # in the stack may be empty, so those ones also should be popped out, because we cannot 
    # goto anywhere else from there  
    while(stack[-1]['temp_set'].empty) :
            stack.pop()
    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

### ALGORITHM

In [90]:
# Decorators 
def exception_handler(func):
    def wrapper(stack):
        try:
            return func(stack)
        except Exception as e:
            # removeLegs(start_leg)
            start_legs.drop(0, axis=0, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            print(f"Error occured = {e}")
            return False
    return wrapper

In [91]:
@exception_handler
def generateTrip(stack):
    # Making the break flag False initially 
    # SHOULD BE CONSIDERED GLOBALLY 
    IS_BREAK = False
    spread_over = 0
    
    while len(stack) > 0 : 
        temp_set = generateTempSet(stack[-1]['current_leg'])
        if temp_set.empty :
            # EXCEPTION CASES CAN BE HANDLED HERE , MEANS WE CAN CHECK WHETHER THE 
            # STACK GETS EMPTY ON BACKTRACKING TO AVOID OUT OF BOUND 
            backtrack(stack)
            # After backtracking a new leg is replaced on the top , so we can just continue
            continue
        
        if (stack[-1]['current_leg'])['Duty'] > MIN_LIMIT_DUTY and stack[-1]["break"] == False:
            # display(stack[-1])
            temp_set = generateTempSet(stack[-1]['current_leg'], 1)
            # temp_set.reset_index(drop=True, inplace=True)
            # print("TempSet is below ")
            # display(temp_set)
            stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set), "break": True})
            spread_over = (stack[-1]['current_leg'])['Arrival Time'] - (stack[0]['current_leg'])['Departure Time']
        else:
            # In the below line , break is made to follow the stack top because after setting it true inside the 
            # break insertion logic , the subsequent legs should have break True . So it should follow stack top  
            stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set), "break": stack[-1]["break"]})
            spread_over = (stack[-1]['current_leg'])['Arrival Time'] - (stack[0]['current_leg'])['Departure Time']
        if (stack[-1]['current_leg'])['Duty'] > MAX_DUTY or spread_over > MAX_SPREAD_OVER \
            or ((stack[-1]['current_leg'])['Duty'] > BREAK_LIMIT_DUTY and stack[-1]["break"] == False) :
            backtrack(stack)
        else:
            if (top_leg:=stack[-1]['current_leg'])['Arrival Place']==START_PLACE and top_leg['Duty'] > MIN_DUTY \
            and stack[-1]["break"]==True: 
                # break
                return stack

### MAIN PART

In [92]:
def main():
    global success, trips_count
    success = 0
    trips_count = 0
    # start_legs = df[df['Departure Place'] == START_PLACE].sort_values('Departure Time', ascending=True)
    # start_legs.reset_index(drop=True, inplace=True)
    # # setting the initial running time of start legs as its running time itself
    # start_legs['Duty'] = start_legs['Running Time']
    while not start_legs.empty:
        # Initializing stack with start leg . stack contains the legs in the trip
        status = generateTrip([{"current_leg": start_legs.iloc[0], "temp_set": pd.DataFrame(), "break": False}])
        if status:
            result = displayTrip(status)
            removeLegs(result[0])
            display(result[0])
            print("Steering Duty = ", result[1])
            print("Spread Over = ", result[2])
        # else means got an exception
        else:
            continue
    print(f"Success legs = {success} ..... Missing {104 - success} routes  ")
    print(f"NUMBER OF TRIPS = {trips_count}")
    print("length of df: ",len(df))

In [93]:
# # MAIN
pd.options.mode.chained_assignment = None #Suppress the warnings 

x = time.time()
if __name__ == "__main__":
    main()
y = time.time()
print(f"Time = {y - x}")
print(f"Time taken by tempset = {exe_generateTempSet}")
print(f"Time taken by popTempSet = {exe_popTempSet}")
x = time.time()

Error occured = single positional indexer is out-of-bounds
Error occured = single positional indexer is out-of-bounds


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time,KM,Running Time,Terminal Gap
0,4,16200,PSL,NH,KLKV,16800,3.5,600,600
1,129,17400,KLKV,NH,TVM,21300,33.7,3900,600
2,317,21900,TVM,NH,KLKV,26400,33.7,4500,600
3,163,27000,KLKV,NH,TVM,31800,33.7,4800,1800
4,360,33600,TVM,NH,NTA,36600,20.7,3000,900
5,212,37500,NTA,PTM,MC,41700,27.7,4200,300
6,432,42000,MC,NH,KLKV,48600,40.0,6600,300
7,88,48900,KLKV,NH,PSL,49500,3.5,600,None


Steering Duty =  08:30:00
Spread Over =  09:15:00
Error occured = single positional indexer is out-of-bounds
Error occured = list index out of range
Error occured = single positional indexer is out-of-bounds


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time,KM,Running Time,Terminal Gap
0,623,18600,PSL,KRKM-MYL-KTDA,TVM,27000,57.0,8400,300
1,599,27300,TVM,AYRA,CHVLA,32700,37.0,5400,1800
2,596,34500,CHVLA,AYRA,MC,41400,44.0,6900,1200
3,407,42600,MC,NH,KLKV,48600,40.0,6000,600
4,73,49200,KLKV,NH,PSL,49800,3.5,600,None


Steering Duty =  07:50:00
Spread Over =  08:40:00
Error occured = single positional indexer is out-of-bounds


...

Steering Duty =  07:45:00
Spread Over =  09:05:00
Error occured = list index out of range


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time,KM,Running Time,Terminal Gap
0,57,49200,PSL,NH,KLKV,49800,3.5,600,1500
1,248,51300,KLKV,NH,TVM,56100,33.7,4800,600
2,439,56700,TVM,NH,NTA,59700,20.7,3000,300
3,276,60000,NTA,NH,TVM,63000,20.7,3000,600
4,466,63600,TVM,NH,KLKV,68400,33.7,4800,1800
5,309,70200,KLKV,NH,TVM,75000,33.7,4800,600
6,498,75600,TVM,NH,KLKV,80100,33.7,4500,300
7,120,80400,KLKV,NH,PSL,81000,3.5,600,None


Steering Duty =  07:55:00
Spread Over =  08:50:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time,KM,Running Time,Terminal Gap
0,56,49500,PSL,NH,KLKV,50100,3.5,600,1200
1,91,51300,KLKV,NH,PSL,51900,3.5,600,900
2,63,52800,PSL,NH,KLKV,53400,3.5,600,300
3,254,53700,KLKV,NH,CSTN,61200,42.0,7500,600
4,453,61800,CSTN,NH,KLKV,69600,42.0,7800,1800
5,304,71400,KLKV,NH,TVM,76200,33.7,4800,600
6,494,76800,TVM,NH,KLKV,81600,33.7,4800,600
7,121,82200,KLKV,NH,PSL,82800,3.5,600,None


Steering Duty =  08:10:00
Spread Over =  09:15:00
Error occured = list index out of range
Error occured = list index out of range
Success legs = 50 ..... Missing 54 routes  
NUMBER OF TRIPS = 38
length of df:  436
Time = 46.85250759124756
Time taken by tempset = 30.067498207092285
Time taken by popTempSet = 14.001464366912842
